In [1]:
import vcfpy
import os
import pandas as pd

# Quality filtering of mutations

In [2]:
work_dir="/mnt/external.data/MeisterLab/Kalyan/EMS_sequencing_data/"
out_dir=os.path.join(work_dir,"QualityFilt")

vcf_dirs =[d for d in os.listdir(os.path.join(work_dir,"annotation/freebayes")) if os.path.isdir(os.path.join(work_dir,"annotation/freebayes",d))]
for vcf_dir in vcf_dirs[0:1]:
    vcf_path=os.path.join(work_dir+"annotation/freebayes",vcf_dir,vcf_dir+".freebayes.filtered.norm.sorted_snpEff.ann.vcf.gz")
    print(vcf_path)
    reader = vcfpy.Reader.from_path(vcf_path)
    os.makedirs(os.path.join(out_dir,vcf_dir), exist_ok=True)
    out_file=os.path.join(out_dir,vcf_dir,vcf_dir+".freebayes.filtered.norm.sorted_snpEff.ann.vcf.gz")
    # with vcfpy.Writer.from_path(out_file, reader.header) as writer:
    #     for record in reader:
    #         ref = record.REF
    #         alt = record.ALT[0].value
            
    #         if (ref == 'G' and alt == 'A') or (ref == 'C' and alt == 'T'):
    #             writer.write_record(record)


/mnt/external.data/MeisterLab/Kalyan/EMS_sequencing_data/annotation/freebayes/EMS_1_CROSS_vs_PMW1074/EMS_1_CROSS_vs_PMW1074.freebayes.filtered.norm.sorted_snpEff.ann.vcf.gz


In [ ]:
reader


TypeError: 'Reader' object is not subscriptable

# Filter only EMS mutations

Used for plotting allele freq along chromosome.

In [ ]:
work_dir="/mnt/external.data/MeisterLab/Kalyan/EMS_sequencing_data/"
out_dir=os.path.join(work_dir,"EMSmutFilt")


vcf_dirs =[d for d in os.listdir(os.path.join(work_dir,"annotation/freebayes")) if os.path.isdir(os.path.join(work_dir,"annotation/freebayes",d))]
for vcf_dir in vcf_dirs[0:3]:
    vcf_path=os.path.join(work_dir+"annotation/freebayes",vcf_dir,vcf_dir+".freebayes.filtered_snpEff.ann.vcf")
    print(vcf_path)
    reader = vcfpy.Reader.from_path(vcf_path)
    os.makedirs(os.path.join(out_dir,vcf_dir), exist_ok=True)
    out_file=os.path.join(out_dir,vcf_dir,vcf_dir+".freebayes.filtered_snpEff.ann.vcf")
    with vcfpy.Writer.from_path(out_file, reader.header) as writer:
        for record in reader:
            ref = record.REF
            alt = record.ALT[0].value
            
            if (ref == 'G' and alt == 'A') or (ref == 'C' and alt == 'T'):
                writer.write_record(record)


# Create CSV with allele freq only for EMS mutations

Used for plotting frequency along genome

In [ ]:
work_dir="/mnt/external.data/MeisterLab/Kalyan/EMS_sequencing_data/TvNmode/"
out_dir=os.path.join(work_dir,"EMSmutFilt")

#  Extract VCF data into a table

vcf_dirs =[d for d in os.listdir(os.path.join(work_dir,"annotation/freebayes")) if os.path.isdir(os.path.join(work_dir,"annotation/freebayes",d))]

for vcf_dir in vcf_dirs[0:3]:
    data = []
    vcf_path = os.path.join(work_dir, "annotation/freebayes", vcf_dir, vcf_dir + ".freebayes.filtered_snpEff.ann.vcf")
    reader = vcfpy.Reader.from_path(vcf_path)
    
    for record in reader:
        ref = record.REF
        alt = record.ALT[0].value if record.ALT else "."
        sample_names = reader.header.samples.names
        
        # Only process EMS mutations
        if (ref == 'G' and alt == 'A') or (ref == 'C' and alt == 'T'):
            # Extract AD field from first sample
            ad = [call.data.get("AD") or ".,." for call in record.calls]
            
            data.append({
                'CHROM': record.CHROM,
                'POS': record.POS,
                'REF': ref,
                'ALT': alt,
                sample_names[0]+"_refCount": ad[0][0],
                sample_names[0]+"_altCount": ad[0][1],  
                sample_names[1]+"_refCount": ad[1][0],
                sample_names[1]+"_altCount": ad[1][1]
            })
    df = pd.DataFrame(data)
    print(f"\nTotal EMS mutations: {len(df)}")
    print(df.head(10))
    df.to_csv(os.path.join(out_dir, vcf_dir+"_EMS_mutation_counts.csv"), index=False)




# Create CSV with allele freq for all SNPs with good MQ

Used for plotting frequency along genome

In [4]:
work_dir="/mnt/external.data/MeisterLab/Kalyan/EMS_sequencing_data/"
out_dir=os.path.join(work_dir,"MQsnpFilt")
os.makedirs(out_dir, exist_ok=True)

#  Extract VCF data into a table

vcf_dirs =[d for d in os.listdir(os.path.join(work_dir,"annotation/freebayes")) if os.path.isdir(os.path.join(work_dir,"annotation/freebayes",d))]

for vcf_dir in vcf_dirs[3:5]:
    data = []
    vcf_path = os.path.join(work_dir, "annotation/freebayes", vcf_dir, vcf_dir + ".freebayes.filtered_snpEff.ann.vcf")
    reader = vcfpy.Reader.from_path(vcf_path)
    
    for record in reader:
        ref = record.REF
        alt = record.ALT[0].value if record.ALT else "."
        sample_names = reader.header.samples.names
        # Extract AD field from first sample
        ad = [call.data.get("AD") or ".,." for call in record.calls]
        # extract mutation type
        type = record.INFO["TYPE"]

        if (record.INFO["TYPE"][0]=="snp"):
            # extract mean quality scores for alt and ref alleles
            MQM = record.INFO.get("MQM", 0)
            MQMR = record.INFO.get("MQMR", 0)
            if MQM[0] >=30 and MQMR >=30:
                data.append({
                    'CHROM': record.CHROM,
                    'POS': record.POS,
                    'REF': ref,
                    'ALT': alt,
                    'MQM': MQM[0],
                    'MQMR': MQMR,
                    sample_names[0]+"_refCount": ad[0][0],
                    sample_names[0]+"_altCount": ad[0][1]
                })

    df = pd.DataFrame(data)
    print(f"\nTotal high quality snps: {len(df)}")
    print(df.head(10))
    df.to_csv(os.path.join(out_dir, vcf_dir+"_high_quality_snp_counts.csv"), index=False)


Total high quality snps: 1634
  CHROM     POS REF ALT   MQM  MQMR  tm580_PMW1074_refCount  \
0     I  229065   C   T  60.0  60.0                      14   
1     I  229117   T   C  60.0  60.0                      19   
2     I  229124   C   T  60.0  60.0                      22   
3     I  229174   A   G  60.0  60.0                      22   
4     I  229204   A   T  60.0  60.0                      17   
5     I  229214   G   A  60.0  60.0                      19   
6     I  229296   G   A  60.0  60.0                      13   
7     I  229382   G   C  60.0  60.0                      14   
8     I  229430   A   G  60.0  60.0                      16   
9     I  229442   A   C  60.0  60.0                      19   

   tm580_PMW1074_altCount  
0                      18  
1                      15  
2                      14  
3                      19  
4                      18  
5                      22  
6                      24  
7                      27  
8                      

In [56]:
len(data)

2102

In [64]:
reader = vcfpy.Reader.from_path(vcf_path)

IncorrectVCFFormat: Missing line starting with "#CHROM"